In [355]:
# import dependencies
import requests
import json
import pandas as pd 
from apikeys import us_state_abbrev

In [356]:
# Read in Global CSVs to Dataframe
nations_disease_df = pd.read_csv('../Resources/disease_nation.csv')

In [357]:
# Clean Global Data
year = 2019
diseases_2019 = nations_disease_df.loc[nations_disease_df['Year'] == year]
diseases_2019 = diseases_2019.rename(columns={'DALYs (Disability-Adjusted Life Years) - Cardiovascular diseases - Sex: Both - Age: All Ages (Number)':'Percent with Cardiovascular Disease',
"DALYs (Disability-Adjusted Life Years) - Diabetes and kidney diseases - Sex: Both - Age: All Ages (Number)":'Percent with Diabetes & Kidney Disease',
'DALYs (Disability-Adjusted Life Years) - Nutritional deficiencies - Sex: Both - Age: All Ages (Number)':'Percent with Nutritional Deficiencies',
"DALYs (Disability-Adjusted Life Years) - Neoplasms - Sex: Both - Age: All Ages (Number)":'Percent with Cancer',
'DALYs (Disability-Adjusted Life Years) - Chronic respiratory diseases - Sex: Both - Age: All Ages (Number)':'Percent with Chronic Respiratory Diseases',
})
diseases_2019.head()
drop_columns=[]
for column in diseases_2019.columns:
    if 'DALYs' in column:
        drop_columns.append(column)
diseases_2019 = diseases_2019.drop(columns=drop_columns)
diseases_2019.replace(['South Sudan','United States','Democratic Republic of Congo','Dominican Republic','Timor',"Cote d'Ivoire",'Central African Republic','Equatorial Guinea','Eswatini','Solomon Islands','Bosnia and Herzegovina'],
['S. Sudan','United States of America','Dem. Rep. Congo','Dominican Rep.','Timor-Leste',"Côte d'Ivoire",'Central African Rep.','Eq. Guinea','eSwatini','Solomon Is.','Bosnia and Herz.'],inplace = True)
diseases_2019 = diseases_2019.dropna(subset=['Code'])


In [358]:
# Bring in population data
world_pop = pd.read_csv('../Resources/populations2019.csv')
world_pop.rename(columns={'Country/Area Name':'Entity'},inplace=True)
world_pop.replace(['South Sudan','United States','Democratic Republic of Congo','Dominican Republic','Timor',"Cote d'Ivoire",'Central African Republic','Equatorial Guinea','Eswatini','Solomon Islands','Bosnia and Herzegovina'],
['S. Sudan','United States of America','Dem. Rep. Congo','Dominican Rep.','Timor-Leste',"Côte d'Ivoire",'Central African Rep.','Eq. Guinea','eSwatini','Solomon Is.','Bosnia and Herz.'],inplace = True)
# diseases_2019 = pd.merge(diseases_2019,world_pop,on = 'Entity')
print(world_pop.loc[world_pop['Entity'] == 'United States'])

In [359]:
# Converting cases to percentages based on population
for column in diseases_2019.columns:
    if 'Percent' in column:
        diseases_2019[column] = diseases_2019[column]/diseases_2019['Population']*100
diseases_2019 = diseases_2019.iloc[:,[0,3,4,5,6,7,8]]
diseases_2019.head()

KeyError: 'Population'

In [ ]:
# Clean US Data
us_cancer = pd.read_csv('../Resources/cancer_by_state.csv')
us_cancer = us_cancer.iloc[:,[0,6,7,8]]
us_cancer.rename(columns={'Area':'State','Age-Adjusted Rate':'Cancer Rate per 100,000 People'}, inplace = True)
us_cancer = us_cancer.drop(columns=['Case Count'])

us_copd = pd.read_csv('../Resources/copd_by_state.csv')
us_copd = us_copd.iloc[:,[0,1,2]]
us_copd.rename(columns={'State Name':'State','Age-Adjusted Prevalence (%)': 'Prevalance of Lung Disease (%)'}, inplace = True)

us_diabetes = pd.read_csv('../Resources/diabetes_by_state.csv')
us_diabetes = us_diabetes.iloc[:,[0,1]]
us_diabetes.rename(columns={'State or US Territory':'State','Percentage of Adults 18 or Older With Diagnosed Diabetes':'% of Adults with Diabetes'}, inplace = True)

us_cardio = pd.read_csv('../Resources/heart_by_state.csv')
us_cardio = us_cardio.loc[us_cardio['YEAR'] == 2019]
us_cardio = us_cardio.iloc[:,[1,2,3]]
us_cardio.rename(columns={'STATE':'State','RATE':'Heart Disease per 100,000 People (2019)'}, inplace = True)
us_cardio.replace({'State':us_state_abbrev},inplace = True)
us_cardio = us_cardio.drop(columns=['DEATHS'])

In [ ]:
# Merge US Data
us_diseases = pd.merge(us_cancer,us_copd,on='State')
us_diseases = pd.merge(us_diseases,us_diabetes,on='State')
us_diseases= pd.merge(us_diseases,us_cardio,on='State')
us_diseases = us_diseases.iloc[:,[0,2,3,1,4,5,6]]
us_diseases.head(2)

,State,Population,State Abbreviation,"Cancer Rate per 100,000 People",Prevalance of Lung Disease (%),% of Adults with Diabetes,"Heart Disease per 100,000 People (2019)"
0,Nevada,3091000,NV,449.4,6.5,9.6,198.1
1,Arizona,7291843,AZ,358.7,5.4,9.6,134.0


In [ ]:
# Read in Houston CSVs
houston_heart = pd.read_csv('../Resources/houston_heart.csv')
houston_copd = pd.read_csv('../Resources/houston_copd.csv')
houston_diabetes = pd.read_csv('../Resources/houston_diabetes.csv')
houston_mental = pd.read_csv('../Resources/houston_mental.csv')
houston_cancer = pd.read_csv('../Resources/houston_cancer.csv')

In [ ]:
# Drop extra columns
houston_cancer.drop(columns=['Unnamed: 2'], inplace=True)
houston_copd.drop(columns=['Unnamed: 2'], inplace=True)
houston_mental.drop(columns=['Unnamed: 2'], inplace=True)

In [ ]:
# Merging Houston Disease Data
houston_disease = pd.merge(houston_cancer,houston_copd,on='Zip Code', how='outer')
houston_disease = pd.merge(houston_disease,houston_diabetes,on='Zip Code', how='outer')
houston_disease = pd.merge(houston_disease,houston_heart,on='Zip Code', how='outer')
houston_disease = pd.merge(houston_disease,houston_mental,on='Zip Code', how='outer')
houston_disease.head()

,Zip Code,% with Cancer,% with COPD,% with Diabetes,% with Coronary Heart Disease,% with Poor Mental Health for 14+ Days
0,77002,2.80%,3.50%,7.00%,3.00%,13.60%
1,77054,2.80%,2.80%,6.20%,2.30%,13.20%
2,77081,2.80%,5.40%,11.60%,4.30%,18.20%
3,77060,3.20%,7.50%,14.40%,5.60%,20.90%
4,77067,3.30%,5.90%,12.90%,4.40%,18.10%


In [ ]:
# Remove % signs and convert to float
houston_disease['% with Cancer'] = houston_disease['% with Cancer'].str.replace('%','').astype(float)
houston_disease['% with COPD'] = houston_disease['% with COPD'].str.replace('%','').astype(float)
houston_disease['% with Diabetes'] = houston_disease['% with Diabetes'].str.replace('%','').astype(float)
houston_disease['% with Coronary Heart Disease'] = houston_disease['% with Coronary Heart Disease'].str.replace('%','').astype(float)
houston_disease['% with Poor Mental Health for 14+ Days'] = houston_disease['% with Poor Mental Health for 14+ Days'].str.replace('%','').astype(float)

In [ ]:
# Read in and merge partner data
us_income = pd.read_csv('../new_csv/states.csv')
us_income = us_income.iloc[:,[1,4]]
us_income.rename(columns={'2019':'Median Household Income'},inplace=True)
usa = pd.merge(us_diseases,us_income,on='State')
usa['Median Household Income'] = usa['Median Household Income'].str.replace(',','').astype(float)
usa.head()


,State,Population,State Abbreviation,"Cancer Rate per 100,000 People",Prevalance of Lung Disease (%),% of Adults with Diabetes,"Heart Disease per 100,000 People (2019)",Median Household Income
0,Nevada,3091000,NV,449.4,6.5,9.6,198.1,63300.0
1,New Mexico,2099634,NM,366.5,4.3,11.2,158.2,51900.0
2,California,39437610,CA,397.0,5.1,9.7,136.9,80400.0
3,Oregon,4216116,OR,411.3,5.1,10.0,131.0,67100.0
4,Alaska,733603,AK,418.5,4.6,8.7,129.7,75500.0


In [ ]:
global_deficiency = pd.read_csv('../Nutrient Deficiency Data/undernourishment_df_world.csv')
global_deficiency = global_deficiency.loc[global_deficiency['Year'] == 2019,:]
global_deficiency['Prevalence of undernourishment (% of population)'] = round(global_deficiency['Prevalence of undernourishment (% of population)'],2)
# diseases_2019 = pd.merge(diseases_2019,global_deficiency,on='Entity')
diseases_2019.head()

In [ ]:
diseases_2019.drop(['Code','Year'],axis=1, inplace=True)
iron_def = pd.read_csv('../Resources/iron_def_children.csv',encoding='latin-1')
iron_def.head()
diseases_2019 = pd.merge(diseases_2019,iron_def,on='Entity')
diseases_2019.head()

,Entity,Code,Year,Percent with Cardiovascular Disease,Percent with Diabetes & Kidney Disease,Percent with Nutritional Deficiencies,Percent with Cancer,Percent with Chronic Respiratory Diseases,% of Children with Iron Deficiency
0,Afghanistan,AFG,2019,1819607.22,476195.52,264468.41,796310.89,306360.72,44.9
1,Albania,ALB,2019,221259.93,25818.06,6697.16,111719.01,19951.39,30.9
2,Algeria,DZA,2019,2053640.89,586525.50,111700.01,675351.37,267334.90,34.3
3,Andorra,AND,2019,2612.38,861.67,87.70,5012.11,1034.13,14.1
4,Angola,AGO,2019,740922.26,280820.29,376306.84,437545.22,189114.06,62.4


In [ ]:
global_income = pd.read_csv('../new_csv/countries.csv')
global_income = global_income.iloc[:,[1,7]]
global_income.rename(columns={'2019':'Median Household Income','Country':'Entity'},inplace=True)
global_income.head()

,Entity,Median Household Income
0,Aruba,NaN
1,Afghanistan,530.0
2,Angola,2970.0
3,Albania,5220.0
4,Andorra,NaN


In [ ]:

diseases_2019 = pd.merge(diseases_2019,global_income,on='Entity')

In [ ]:
diseases_2019.head()

,Entity,Code,Year,Percent with Cardiovascular Disease,Percent with Diabetes & Kidney Disease,Percent with Nutritional Deficiencies,Percent with Cancer,Percent with Chronic Respiratory Diseases,% of Children with Iron Deficiency
0,Afghanistan,AFG,2019,1819607.22,476195.52,264468.41,796310.89,306360.72,44.9
1,Albania,ALB,2019,221259.93,25818.06,6697.16,111719.01,19951.39,30.9
2,Algeria,DZA,2019,2053640.89,586525.50,111700.01,675351.37,267334.90,34.3
3,Andorra,AND,2019,2612.38,861.67,87.70,5012.11,1034.13,14.1
4,Angola,AGO,2019,740922.26,280820.29,376306.84,437545.22,189114.06,62.4


In [ ]:
# Write dataframes to CSV files for analysis and visualization
houston_disease.to_csv('../Output/csv/houston_disease.csv')
usa.to_csv('../Output/csv/us_combined.csv')
diseases_2019.to_csv('../Output/csv/global_combined.csv')
